# Start to end test

In [ ]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [ ]:
# Import all necessary libraries.
import cv2
import numpy as np
import sys
import matplotlib.pyplot as plt
from termcolor import colored

plt.rcParams["figure.figsize"] = (10,10)

In [ ]:
# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [ ]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convert_cv_zones_rgb_to_bgr, reshape_points
from NomeroffNet.MultiLineNPExtractor import CCraft
                                            

ccraft = CCraft()
npPointsCraft = NpPointsCraft()
npPointsCraft.load()

In [ ]:
from NomeroffNet.YoloV5Detector import Detector

detector = Detector()
detector.load()

In [ ]:
from NomeroffNet.OptionsDetector import OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load()

In [ ]:
from NomeroffNet.TextDetector import TextDetector
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "latest"
    }
})

In [ ]:
async def test(dirName, fname, y, min_bbox_acc = 0.5, verbose=0):
    nGood = 0
    nBad  = 0
    img_path = os.path.join(dirName, fname)
    if verbose==1:
        print(colored(f"__________ \t\t {img_path} \t\t __________", "blue"))
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    targetBoxes = detector.detect_bbox(img)
    
    # all_points = npPointsCraft.detect(img, targetBoxes, [5,2,0])
    all_points, all_mline_boxes = npPointsCraft.detect_mline(img, targetBoxes)
    # for  images/14.jpeg bug
    all_points = [ps for ps in all_points if len(ps)]

    
    print('all_points')
    print(all_points)
    # cut zones
    toShowZones = [getCvZoneRGB(img, reshape_points(rect,1)) for rect in all_points]
    zones = convert_cv_zones_rgb_to_bgr(toShowZones)    

    # find standart
    regionIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    print(regionNames)
    print(countLines)
    
    # convert multiline to one line
    image_parts = [img[int(box[1]):int(box[3]), int(box[0]):int(box[2])] 
                   for box, cl in zip(targetBoxes, countLines) 
                   if cl > 1]
    all_mline_boxes_rect = [mline_boxes 
                   for mline_boxes, cl in zip(all_mline_boxes, countLines) 
                   if cl > 1]
    region_names_rect = [region_name 
                           for region_name, cl in zip(regionNames, countLines) 
                           if cl > 1]
    index_rect = [i 
                  for i, cl in enumerate(countLines) 
                  if cl > 1]
    (zones_rect, 
     zones_target_points, 
     zones_mline_boxes) = ccraft.multiline_to_one_line(all_mline_boxes_rect,
                                                            image_parts,
                                                            region_names_rect)
    for i, zone in zip(index_rect, zones_rect):
        zones[i] = zone
    
    
    for zone, points in zip(toShowZones, all_points):
        plt.axis("off")
        plt.imshow(zone)
        plt.show()

    # find text with postprocessing by standart  
    textArr = textDetector.predict(zones, regionNames, countLines)
    print(textArr)
    
     # draw rect and 4 points
    for targetBox, points in zip(targetBoxes, all_points):
        # draw 
        cv2.rectangle(img, 
                      (int(targetBox[0]), int(targetBox[1])), 
                      (int(targetBox[2]), int(targetBox[3])), 
                      (0,120,255), 
                      3)
        #print(points, points.shape)
        cv2.polylines(img, np.array([points], np.int32), True, (255,120,255),3)
    plt.imshow(img)
    plt.show()

    for yText in y:
        if yText in textArr:
            print(colored(f"OK: TEXT:{yText} \t\t\t RESULTS:{textArr} \n\t\t\t\t\t in PATH:{img_path}", 'green'))
            nGood += 1
        else:
            print(colored(f"NOT OK: TEXT:{yText} \t\t\t RESULTS:{textArr} \n\t\t\t\t\t in PATH:{img_path} ", 'red'))
            nBad += 1
    return nGood, nBad

In [ ]:
dirName = "./images"

In [ ]:
testData = {
    "0.jpeg": ["AI5255EI"],
    "1.jpeg": ["AT6883CM"],
    "2.jpeg": ["AT1515CK"],
    "3.jpeg": ["BX0578CE"],
    "4.jpeg": ["AC4249CB"],
    "5.jpeg": ["BC3496HC"],
    "6.jpeg": ["BC3496HC"],
    "7.jpeg": ["AO1306CH"],
    "8.jpeg": ["AE1077CO"],
    "9.jpeg": ["AB3391AK"],
    "10.jpeg": ["BE7425CB"],
    "11.jpeg": ["BE7425CB"],
    "12.jpeg": ["AB0680EA"],
    "13.jpeg": ["AB0680EA"],
    "14.jpeg": ["BM1930BM"],
    "15.jpeg": ["AI1382HB"],
    "16.jpeg": ["AB7333BH"],
    "17.jpeg": ["AB7642CT"],
    "18.jpeg": ["AC4921CB"],
    "19.jpeg": ["BC9911BK"],
    "20.jpeg": ["BC7007AK"],
    "21.jpeg": ["AB5649CI"],
    "22.jpeg": ["AX2756EK"],
    "23.jpeg": ["AA7564MX"],
    "24.jpeg": ["AM5696CK"],
    "25.jpeg": ["AM5696CK"],
}

%matplotlib inline
plt.rcParams["figure.figsize"]  = (20, 20)    

gGood = 0
gBad = 0
i = 0
for fileName in testData.keys():
    nGood, nBad = await test(dirName, fileName, testData[fileName], verbose=1)
    gGood += nGood
    gBad += nBad
    i += 1
total = gGood + gBad
print(f"TOTAL GOOD: {gGood/total}")
print(f"TOTAL BED: {gBad/total}")